In [2]:
import sys
sys.path.append(r"E:\Coding\Intern Sprouts\sign laguange detection\yolov5")
model_path=r"E:\Coding\Intern Sprouts\sign laguange detection\yolov5\runs\train\exp4\weights\best.pt"

1. Install necessary libraries

In [3]:
import cv2
import torch
from yolov5.models.common import DetectMultiBackend
from yolov5.utils.general import non_max_suppression, scale_boxes

2. Setup yolov5s model

In [4]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=DetectMultiBackend(model_path,device=device)
model.eval()

Fusing layers... 
Model summary: 157 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs


DetectMultiBackend(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (2): C3(
        (cv1): Conv(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv3): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (m): Sequential(
          (0): Bottleneck(
            (cv1): Conv(
              (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
              (act): SiLU(inplace=True)
            )
            (cv2): Conv(

3. Define class names

In [5]:
class_names=["hello","iloveyou","no","thanks","yes"]

4. Real time video captioning

In [7]:
def iou(box1, box2):
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])

    inter_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    return inter_area / float(box1_area + box2_area - inter_area + 1e-6)


cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    sys.exit()

def draw_bounding_box(img, box, label, color=(0, 0, 255), thickness=2):
    x1, y1, x2, y2 = map(int, box)
    cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness)
    cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, thickness)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break
    
    img = cv2.resize(frame, (640, 640))
    img_rgb = img[..., ::-1]  # Convert BGR to RGB
    
    img_tensor = torch.from_numpy(img_rgb.copy()).float().to(device).permute(2, 0, 1).unsqueeze(0) / 255.0

    results = model(img_tensor, augment=False)  
    nms_conf_threshold = 0.4 
    detections = non_max_suppression(results, conf_thres=nms_conf_threshold, iou_thres=0.5)[0] 

    if detections is not None and len(detections):
        detections[:, :4] = scale_boxes(img_tensor.shape[2:], detections[:, :4], frame.shape).round()
        boxes = []
        for *xyxy, conf, cls in detections:
            box = xyxy  
            if not any(iou(box, existing_box) > 0.5 for existing_box in boxes): 
                boxes.append(box)  
                label = f"{class_names[int(cls)]} {conf:.2f}"
                draw_bounding_box(frame, box, label)  
    
    cv2.imshow('Sign Language Detection', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
